In [1]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
batch_size = 32
num_classes = 10
epochs = 1
data_augmentation = True
num_predictions = 20

In [4]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'тренировочные примеры')
print(x_test.shape[0], 'тестовые примеры')

x_train shape: (50000, 32, 32, 3)
50000 тренировочные примеры
10000 тестовые примеры


In [5]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [6]:
model = Sequential()

In [7]:
# Оставили один блок (conv-pool)
# уменьшили kernel_size до (2, 2)
# увеличили число фильтров втрое
# 'углубили' нейронную сеть
model.add(Conv2D(96, (2, 2), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(96, (2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(96, (2, 2)))
model.add(Activation('sigmoid')) # добавили
model.add(Conv2D(96, (2, 2)))
model.add(Activation('tanh')) # добавили
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

In [8]:
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [9]:
opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)

In [10]:
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [11]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [12]:
if not data_augmentation:
    print('Не используется data augmentation')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Использование data augmentation в реальном времени')
    datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False, 
        zca_epsilon=1e-06, 
        rotation_range=0, 
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0., 
        zoom_range=0., 
        channel_shift_range=0.,
        fill_mode='nearest',
        cval=0.,
        horizontal_flip=True,
        vertical_flip=False,
        rescale=None,
        preprocessing_function=None,
        data_format=None,
        validation_split=0.0)

Использование data augmentation в реальном времени


In [13]:
model.fit_generator(datagen.flow(x_train, y_train,
                        batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

1563/1563 [==============================] - 362s 230ms/step - loss: 1.8693 - accuracy: 0.3315 - val_loss: 1.5630 - val_accuracy: 0.4340


In [14]:
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

313/313 [==============================] - 9s 30ms/step - loss: 1.5630 - accuracy: 0.4340
Test loss: 1.5629873275756836
Test accuracy: 0.4339999854564667


## Замечания:

1) Уменьшение числа блоков conv-pool не оказало существенного изменения
   в качестве;

2) При изменении размера ядра результат немного ухудшился;

3) При увеличении количества фильтров показатели качества выросли;

4) Увеличении глубины сети неожиданно повлекло ухудшение показателей точности.